In [ ]:
import os
import sys
import warnings

warnings.filterwarnings("ignore")

os.chdir('../../')

In [ ]:
pwd

In [ ]:
from gradsflow.autoclassifier import AutoImageClassifier

from flash.core.data.utils import download_data
from flash.image import ImageClassificationData

In [ ]:
datamodule = ImageClassificationData.from_folders(
    train_folder="data/hymenoptera_data/train/",
    val_folder="data/hymenoptera_data/val/",
)

In [ ]:
suggested_conf = dict(
    optimizers=["adam"],
    lr=(5e-4, 1e-3),
)

model = AutoImageClassifier(datamodule,
                            suggested_backbones=['ssl_resnet18'],
                            suggested_conf=suggested_conf,
                            max_epochs=1,
                            optimization_metric="val_accuracy",
                            timeout=5)

print("AutoImageClassifier initialised!")
model.hp_tune()

In [ ]:
model.model